In [ ]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=f3bc40de265ca54bb467edf3a75d6489207bbc545f2cbf4739244e32fd9b2855
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=6a48a44fa556b78e97f50b7baf74d990eb8bbc3153b4dd531891d8574c9d5fff
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
pip install SMOTE

In [ ]:
pip install imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.2 MB/s eta 0:00:00


In [ ]:
pip install joblib

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import ta  # Technical Analysis library
import pandas_ta as pata
import shap
from sklearn.model_selection import train_test_split, cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


######################

**초기 데이터 처리**
######################

In [ ]:
# Load the dataset from Kaggle
file_path = '/content/drive/MyDrive/Data/SOL_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,open_time,open,high,low,close,volume
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91


######################

**1/3 데이터 처리**
######################

In [ ]:
# 데이터의 첫 번째 1/3만 사용
n_samples = len(data)
data = data.iloc[:n_samples // 3]

######################

**가용 지표 입력**
######################

In [ ]:
# open_time 열을 datetime 형식으로 변환
data['open_time'] = pd.to_datetime(data['open_time'])

# 시각과 분만 추출하여 time 열 생성
data['time'] = data['open_time'].dt.strftime('%H:%M')

# ATR 계산
atr_periods = [5, 10, 14, 20, 50]
for period in atr_periods:
    data[f'atr_{period}'] = ta.volatility.average_true_range(data['high'], data['low'], data['close'], window=period)

# VWAP 계산
data['vwap'] = ta.volume.volume_weighted_average_price(data['high'], data['low'], data['close'], data['volume'])

# Stochastic Oscillator 계산
stoch_periods = [(14, 3), (21, 5), (9, 3), (5, 2), (20, 7)]
for period, smooth in stoch_periods:
    data[f'stoch_%k_{period}_{smooth}'] = ta.momentum.stoch(data['high'], data['low'], data['close'], window=period, smooth_window=smooth)
    data[f'stoch_%d_{period}_{smooth}'] = ta.momentum.stoch_signal(data['high'], data['low'], data['close'], window=period, smooth_window=smooth)

# Supertrend 계산 함수
def calculate_supertrend(df, period=7, multiplier=3, atr_period=14):
    df = df.copy()  # 데이터프레임을 복사하여 사용
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=atr_period)
    df['upperband'] = hl2 + (multiplier * df['atr'])
    df['lowerband'] = hl2 - (multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in tqdm(range(1, len(df.index)), desc=f'Calculating Supertrend {period}-{multiplier}-{atr_period}'):
        previous = current - 1

        if df['close'].iloc[current] > df['upperband'].iloc[previous]:
            df.loc[df.index[current], 'in_uptrend'] = True
        elif df['close'].iloc[current] < df['lowerband'].iloc[previous]:
            df.loc[df.index[current], 'in_uptrend'] = False
        else:
            df.loc[df.index[current], 'in_uptrend'] = df['in_uptrend'].iloc[previous]

            if df['in_uptrend'].iloc[current] and df['lowerband'].iloc[current] < df['lowerband'].iloc[previous]:
                df.loc[df.index[current], 'lowerband'] = df['lowerband'].iloc[previous]

            if not df['in_uptrend'].iloc[current] and df['upperband'].iloc[current] > df['upperband'].iloc[previous]:
                df.loc[df.index[current], 'upperband'] = df['upperband'].iloc[previous]

    return df

# Supertrend 계산
supertrend_settings = [(7, 3, 14), (10, 3, 20), (14, 2, 10), (20, 4, 50), (50, 5, 5)]
for period, multiplier, atr_period in supertrend_settings:
    data = calculate_supertrend(data, period, multiplier, atr_period)
    data[f'supertrend_upper_{period}_{multiplier}_{atr_period}'] = data['upperband']
    data[f'supertrend_lower_{period}_{multiplier}_{atr_period}'] = data['lowerband']
    data[f'supertrend_in_uptrend_{period}_{multiplier}_{atr_period}'] = data['in_uptrend']

######################

**비가용 지표**
######################

In [ ]:
# Bollinger Bands 계산
bollinger_periods = [10, 20, 50, 100, 200]
for period in bollinger_periods:
    data[f'bollinger_hband_{period}'] = ta.volatility.BollingerBands(data['close'], window=period).bollinger_hband()
    data[f'bollinger_lband_{period}'] = ta.volatility.BollingerBands(data['close'], window=period).bollinger_lband()

# OBV 계산
data['obv'] = ta.volume.on_balance_volume(data['close'], data['volume'])

# Ichimoku 계산
ichimoku_periods = [9, 26, 52, 100, 200]
for period in ichimoku_periods:
    data[f'ichimoku_base_{period}'] = ta.trend.ichimoku_base_line(data['high'], data['low'], window1=period)
    data[f'ichimoku_conversion_{period}'] = ta.trend.ichimoku_conversion_line(data['high'], data['low'], window1=period)


######################

**60봉 이내 최대 상승 및 하락 %**
######################

In [ ]:
def calculate_max_min_returns(df):
    window_size = 60

    # 'open_time'이 이미 인덱스로 설정되어 있는지 확인
    if df.index.name != 'open_time':
        raise KeyError("'open_time' 열이 인덱스로 설정되어 있어야 합니다.")

    # 인덱스 중복 확인 및 제거
    df = df[~df.index.duplicated(keep='first')].copy()

    # 60분 윈도우를 적용하여 최대 및 최소 가격 계산
    df['max_price'] = df['high'].rolling(f'{window_size}T').max().shift(-window_size)
    df['min_price'] = df['low'].rolling(f'{window_size}T').min().shift(-window_size)

    # 결측값을 적절히 처리 (예: 마지막 몇 행)
    df['max_price'].fillna(df['high'], inplace=True)
    df['min_price'].fillna(df['low'], inplace=True)

    # 현재 가격
    current_price = df['close']

    # 최대 및 최소 수익률 계산
    df['max_return_60min'] = ((df['max_price'] - current_price) / current_price) * 100
    df['min_return_60min'] = ((df['min_price'] - current_price) / current_price) * 100

    # 필요없는 열 삭제
    df.drop(columns=['max_price', 'min_price'], inplace=True)

    return df

# 최대 상승률과 최대 하락률 계산
data = calculate_max_min_returns(data)

In [ ]:
# 인덱스를 리셋하여 'open_time'을 열로 되돌림
data.reset_index(inplace=True)

data.head()

In [ ]:
data.columns

######################

**처리 데이터 저장**
######################

In [ ]:
path2 = "/content/drive/MyDrive/Data/SOL60_INDICATOR2_SMALL";
data.to_csv(path2, index=True);

######################

**처리 데이터 로딩**
######################

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Data/SOL60_INDICATOR2_SMALL")

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,supertrend_lower_14_2_10,supertrend_in_uptrend_14_2_10,supertrend_upper_20_4_50,supertrend_lower_20_4_50,supertrend_in_uptrend_20_4_50,supertrend_upper_50_5_5,supertrend_lower_50_5_5,supertrend_in_uptrend_50_5_5,max_return_60min,min_return_60min
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,0.0,0.0,...,1.44510,True,1.44510,1.44510,True,1.44510,1.44510,True,1.203652,-0.387382
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,0.0,0.0,...,1.44705,True,1.44705,1.44705,True,1.44705,1.44705,True,1.154671,-0.435594
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,0.0,0.0,...,1.44980,True,1.44980,1.44980,True,1.44980,1.44980,True,1.008009,-0.579950
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,0.0,0.0,...,1.45310,True,1.45310,1.45310,True,1.45310,1.45310,True,0.494573,-1.085314
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,0.0,0.0,...,1.45570,True,1.45570,1.45570,True,1.47590,1.43550,True,0.418697,-1.159997


######################

지표 목록 파악

######################



In [ ]:
data.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'time', 'atr_5',
       'atr_10', 'atr_14', 'atr_20', 'atr_50', 'vwap', 'stoch_%k_14_3',
       'stoch_%d_14_3', 'stoch_%k_21_5', 'stoch_%d_21_5', 'stoch_%k_9_3',
       'stoch_%d_9_3', 'stoch_%k_5_2', 'stoch_%d_5_2', 'stoch_%k_20_7',
       'stoch_%d_20_7', 'obv', 'bollinger_hband_10', 'bollinger_lband_10',
       'bollinger_hband_20', 'bollinger_lband_20', 'bollinger_hband_50',
       'bollinger_lband_50', 'bollinger_hband_100', 'bollinger_lband_100',
       'bollinger_hband_200', 'bollinger_lband_200', 'ichimoku_base_9',
       'ichimoku_conversion_9', 'ichimoku_base_26', 'ichimoku_conversion_26',
       'ichimoku_base_52', 'ichimoku_conversion_52', 'ichimoku_base_100',
       'ichimoku_conversion_100', 'ichimoku_base_200',
       'ichimoku_conversion_200', 'atr', 'upperband', 'lowerband',
       'in_uptrend', 'supertrend_upper_7_3_14', 'supertrend_lower_7_3_14',
       'supertrend_in_uptrend_7_3_14', 'supertrend_upper_10

######################

**불필요 지표 삭제**
######################

In [ ]:
data = data.drop(columns=['ichimoku_base_9',
       'ichimoku_conversion_9', 'ichimoku_base_26', 'ichimoku_conversion_26',
       'ichimoku_base_52', 'ichimoku_conversion_52', 'ichimoku_base_100',
       'ichimoku_conversion_100', 'ichimoku_base_200',
       'ichimoku_conversion_200','bollinger_hband_10', 'bollinger_lband_10',
       'bollinger_hband_20', 'bollinger_lband_20', 'bollinger_hband_50',
       'bollinger_lband_50', 'bollinger_hband_100', 'bollinger_lband_100',
       'bollinger_hband_200', 'bollinger_lband_200', 'obv'])

In [ ]:
data.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'time', 'atr_5',
       'atr_10', 'atr_14', 'atr_20', 'atr_50', 'vwap', 'stoch_%k_14_3',
       'stoch_%d_14_3', 'stoch_%k_21_5', 'stoch_%d_21_5', 'stoch_%k_9_3',
       'stoch_%d_9_3', 'stoch_%k_5_2', 'stoch_%d_5_2', 'stoch_%k_20_7',
       'stoch_%d_20_7', 'atr', 'upperband', 'lowerband', 'in_uptrend',
       'supertrend_upper_7_3_14', 'supertrend_lower_7_3_14',
       'supertrend_in_uptrend_7_3_14', 'supertrend_upper_10_3_20',
       'supertrend_lower_10_3_20', 'supertrend_in_uptrend_10_3_20',
       'supertrend_upper_14_2_10', 'supertrend_lower_14_2_10',
       'supertrend_in_uptrend_14_2_10', 'supertrend_upper_20_4_50',
       'supertrend_lower_20_4_50', 'supertrend_in_uptrend_20_4_50',
       'supertrend_upper_50_5_5', 'supertrend_lower_50_5_5',
       'supertrend_in_uptrend_50_5_5', 'max_return_60min', 'min_return_60min'],
      dtype='object')

######################

**추가 지표 처리**
######################

In [ ]:
# 지표 계산 함수 정의

def calculate_indicators(df):
    # Parabolic SAR
    df['Parabolic_SAR_0.02'] = pata.psar(df['high'], df['low'], df['close'], af=0.02, max_af=0.2)['PSARl_0.02_0.2']
    df['Parabolic_SAR_0.04'] = pata.psar(df['high'], df['low'], df['close'], af=0.04, max_af=0.2)['PSARl_0.04_0.2']
    df['Parabolic_SAR_0.06'] = pata.psar(df['high'], df['low'], df['close'], af=0.06, max_af=0.2)['PSARl_0.06_0.2']
    df['Parabolic_SAR_0.08'] = pata.psar(df['high'], df['low'], df['close'], af=0.08, max_af=0.2)['PSARl_0.08_0.2']
    df['Parabolic_SAR_0.1'] = pata.psar(df['high'], df['low'], df['close'], af=0.1, max_af=0.2)['PSARl_0.1_0.2']

    # Williams %R
    df['Williams_%R_10'] = pata.willr(df['high'], df['low'], df['close'], length=10)
    df['Williams_%R_20'] = pata.willr(df['high'], df['low'], df['close'], length=20)
    df['Williams_%R_30'] = pata.willr(df['high'], df['low'], df['close'], length=30)
    df['Williams_%R_40'] = pata.willr(df['high'], df['low'], df['close'], length=40)
    df['Williams_%R_50'] = pata.willr(df['high'], df['low'], df['close'], length=50)

    # Momentum
    df['Momentum_10'] = pata.mom(df['close'], length=10)
    df['Momentum_20'] = pata.mom(df['close'], length=20)
    df['Momentum_30'] = pata.mom(df['close'], length=30)
    df['Momentum_40'] = pata.mom(df['close'], length=40)
    df['Momentum_50'] = pata.mom(df['close'], length=50)

    # Rate of Change (ROC)
    df['ROC_10'] = pata.roc(df['close'], length=10)
    df['ROC_20'] = pata.roc(df['close'], length=20)
    df['ROC_30'] = pata.roc(df['close'], length=30)
    df['ROC_40'] = pata.roc(df['close'], length=40)
    df['ROC_50'] = pata.roc(df['close'], length=50)

    # Chande Momentum Oscillator (CMO)
    df['CMO_10'] = pata.cmo(df['close'], length=10)
    df['CMO_20'] = pata.cmo(df['close'], length=20)
    df['CMO_30'] = pata.cmo(df['close'], length=30)
    df['CMO_40'] = pata.cmo(df['close'], length=40)
    df['CMO_50'] = pata.cmo(df['close'], length=50)

    # Money Flow Index (MFI)
    df['MFI_10'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=10)
    df['MFI_20'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=20)
    df['MFI_30'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=30)
    df['MFI_40'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=40)
    df['MFI_50'] = pata.mfi(df['high'], df['low'], df['close'], df['volume'], length=50)

    # Relative Strength Index (RSI)
    df['RSI_10'] = pata.rsi(df['close'], length=10)
    df['RSI_20'] = pata.rsi(df['close'], length=20)
    df['RSI_30'] = pata.rsi(df['close'], length=30)
    df['RSI_40'] = pata.rsi(df['close'], length=40)
    df['RSI_50'] = pata.rsi(df['close'], length=50)

    # Accumulation/Distribution Line (A/D Line)
    df['Accumulation_Distribution_Line'] = pata.ad(df['high'], df['low'], df['close'], df['volume'])

    # Elder's Force Index (EFI)
    df['Elder_Force_Index_2'] = pata.efi(df['close'], df['volume'], length=2)
    df['Elder_Force_Index_13'] = pata.efi(df['close'], df['volume'], length=13)
    df['Elder_Force_Index_5'] = pata.efi(df['close'], df['volume'], length=5)
    df['Elder_Force_Index_10'] = pata.efi(df['close'], df['volume'], length=10)
    df['Elder_Force_Index_25'] = pata.efi(df['close'], df['volume'], length=25)

    # Relative Vigor Index (RVI)
    df['Relative_Vigor_Index_10'] = pata.rsi(df['close'], length=10)  # pandas-ta는 RVI가 없으므로 RSI로 대체
    df['Relative_Vigor_Index_20'] = pata.rsi(df['close'], length=20)
    df['Relative_Vigor_Index_30'] = pata.rsi(df['close'], length=30)
    df['Relative_Vigor_Index_40'] = pata.rsi(df['close'], length=40)
    df['Relative_Vigor_Index_50'] = pata.rsi(df['close'], length=50)

    # Volume Ratio (VR)
    def volume_ratio(close, volume, period):
        vr = []
        for i in range(len(close)):
            if i < period:
                vr.append(None)
            else:
                vol_up = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] > close[j - 1])
                vol_down = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] < close[j - 1])
                vol_same = sum(volume[j] for j in range(i - period + 1, i + 1) if close[j] == close[j - 1])
                vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
                vr.append(vr_value)
        return vr

    df['VR_10'] = volume_ratio(df['close'], df['volume'], period=10)
    df['VR_20'] = volume_ratio(df['close'], df['volume'], period=20)
    df['VR_30'] = volume_ratio(df['close'], df['volume'], period=30)
    df['VR_40'] = volume_ratio(df['close'], df['volume'], period=40)
    df['VR_50'] = volume_ratio(df['close'], df['volume'], period=50)

    return df

# 지표 계산
data = calculate_indicators(data)

<ipython-input-34-b6fdbbcc9f9d>:80: RuntimeWarning: divide by zero encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-34-b6fdbbcc9f9d>:80: RuntimeWarning: invalid value encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-34-b6fdbbcc9f9d>:80: RuntimeWarning: divide by zero encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-34-b6fdbbcc9f9d>:80: RuntimeWarning: invalid value encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-34-b6fdbbcc9f9d>:80: RuntimeWarning: divide by zero encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<ipython-input-34-b6fdbbcc9f9d>:80: RuntimeWarning: invalid value encountered in scalar divide
  vr_value = (vol_up + vol_same / 2) / (vol_down + vol_same / 2) * 100
<

In [ ]:
data.head()

,open_time,open,high,low,close,volume,time,atr_5,atr_10,atr_14,...,Relative_Vigor_Index_10,Relative_Vigor_Index_20,Relative_Vigor_Index_30,Relative_Vigor_Index_40,Relative_Vigor_Index_50,VR_10,VR_20,VR_30,VR_40,VR_50
0,2020-12-31 15:00:00,1.4458,1.4458,1.4444,1.4456,161.34,15:00,0.00000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-12-31 15:01:00,1.4477,1.4478,1.4463,1.4463,148.86,15:01,0.00000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-12-31 15:02:00,1.4479,1.4530,1.4466,1.4484,3208.13,15:02,0.00000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-12-31 15:03:00,1.4503,1.4559,1.4503,1.4558,1639.76,15:03,0.00000,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-12-31 15:04:00,1.4558,1.4569,1.4545,1.4569,900.91,15:04,0.00404,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'time', 'atr_5',
       'atr_10', 'atr_14', 'atr_20', 'atr_50', 'vwap', 'stoch_%k_14_3',
       'stoch_%d_14_3', 'stoch_%k_21_5', 'stoch_%d_21_5', 'stoch_%k_9_3',
       'stoch_%d_9_3', 'stoch_%k_5_2', 'stoch_%d_5_2', 'stoch_%k_20_7',
       'stoch_%d_20_7', 'atr', 'upperband', 'lowerband', 'in_uptrend',
       'supertrend_upper_7_3_14', 'supertrend_lower_7_3_14',
       'supertrend_in_uptrend_7_3_14', 'supertrend_upper_10_3_20',
       'supertrend_lower_10_3_20', 'supertrend_in_uptrend_10_3_20',
       'supertrend_upper_14_2_10', 'supertrend_lower_14_2_10',
       'supertrend_in_uptrend_14_2_10', 'supertrend_upper_20_4_50',
       'supertrend_lower_20_4_50', 'supertrend_in_uptrend_20_4_50',
       'supertrend_upper_50_5_5', 'supertrend_lower_50_5_5',
       'supertrend_in_uptrend_50_5_5', 'max_return_60min', 'min_return_60min',
       'Parabolic_SAR_0.02', 'Parabolic_SAR_0.04', 'Parabolic_SAR_0.06',
       'Parabolic_SAR_0.

######################

**지표 저장**
######################

In [ ]:
path_edit = "/content/drive/MyDrive/Data/SOL60_INDICATOR3_SMALL";
data.to_csv(path_edit, index=True);

NameError: name 'data' is not defined

######################

**지표 관계 파악(간단)**
######################

In [ ]:
# max_return_60min이 1.1 이상인 데이터 필터링
filtered_df_relative = data[data['max_return_60min'] >= 1.1]

# 'time' 컬럼을 분 단위로 변환
filtered_df_relative['time'] = filtered_df_relative['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))

# 상관계수 계산 (open_time을 제외한 나머지 항목들에 대해서만 계산)
correlation_matrix = filtered_df_relative.drop(columns=['open_time']).corr()

# max_return_60min과 나머지 항목들 간의 상관계수 출력
max_return_correlations = correlation_matrix['max_return_60min'].drop('max_return_60min').sort_values(ascending=False)
max_return_correlations

<ipython-input-38-97ae0006df35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_relative['time'] = filtered_df_relative['time'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))


volume                            0.290336
atr                               0.102573
atr_5                             0.102573
atr_10                            0.099949
atr_14                            0.097475
                                    ...   
supertrend_lower_7_3_14          -0.141304
supertrend_lower_20_4_50         -0.141515
supertrend_lower_50_5_5          -0.143015
lowerband                        -0.143015
Accumulation_Distribution_Line   -0.144703
Name: max_return_60min, Length: 93, dtype: float64

######################

**학습 데이터 처리**
######################

In [ ]:
# open_time 열이 datetime 형식이 아닌 경우 변환
if not np.issubdtype(data['open_time'].dtype, np.datetime64):
    data['open_time'] = pd.to_datetime(data['open_time'])

# time 열을 분 단위로 변환
data['time'] = data['open_time'].dt.hour * 60 + data['open_time'].dt.minute

# 사용하지 않을 열 제외
data = data.drop(columns=['open_time'])

In [ ]:
# 목표 변수 생성 (max_return_60min이 1.1% 이상인 경우 1, 그렇지 않으면 0)
data['target'] = (data['max_return_60min'] >= 1.1).astype(int)

# 특성과 목표 변수 분리
X = data.drop(columns=['max_return_60min', 'min_return_60min', 'target'])
y = data['target']

# 무한대 값을 NaN으로 대체
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 평균으로 대체
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# 데이터 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_imputed)

# SMOTE를 사용한 오버샘플링
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# 시계열 데이터 생성 함수
def create_sequences(X, y, time_steps=60):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32)

# 데이터 크기를 확인하고 time_steps 값 조정
time_steps = 60
if len(X_resampled) <= time_steps:
    raise ValueError(f"The dataset size ({len(X_resampled)}) is too small for the given time_steps ({time_steps}).")

X_sequences, y_sequences = create_sequences(X_resampled, y_resampled, time_steps)

######################

**xgboost**
######################

In [ ]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 4.6 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# XGBoost 모델 정의
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42, use_label_encoder=False, eval_metric='logloss')

# 모델 학습
xgb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred = xgb_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

X_train shape: (607761, 60, 62)
X_test shape: (151941, 60, 62)
y_train shape: (607761,)
y_test shape: (151941,)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:42:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.731
Precision: 0.772
Recall: 0.656
F1 Score: 0.709


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

######################

**catboost**
######################

In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# CatBoost 모델 정의
cat_model = CatBoostClassifier(iterations=100, learning_rate=0.05, depth=6, random_seed=42, verbose=0)

# 모델 학습
cat_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred = cat_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

X_train shape: (607761, 60, 62)
X_test shape: (151941, 60, 62)
y_train shape: (607761,)
y_test shape: (151941,)
Accuracy: 0.733
Precision: 0.769
Recall: 0.665
F1 Score: 0.713


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(cat_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

######################

**lightgbm**
######################

In [ ]:
pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.9 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgb

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# LightGBM 데이터셋 생성
lgb_train = lgb.Dataset(X_train.reshape(X_train.shape[0], -1), y_train, params={'max_bin': 255})
lgb_eval = lgb.Dataset(X_test.reshape(X_test.shape[0], -1), y_test, reference=lgb_train, params={'max_bin': 255})

# LightGBM 하이퍼파라미터 설정
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

# 모델 학습
gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=100,
    valid_sets=lgb_eval,
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# 예측
y_pred_prob = gbm.predict(X_test.reshape(X_test.shape[0], -1), num_iteration=gbm.best_iteration)
y_pred = (y_pred_prob > 0.5).astype(int)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

X_train shape: (607761, 60, 62)
X_test shape: (151941, 60, 62)
y_train shape: (607761,)
y_test shape: (151941,)
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.49074	valid_0's auc: 0.820659
Accuracy: 0.7378719371334926
Precision: 0.7856690319310694
Recall: 0.6535441370223979
F1 Score: 0.7135418165079548


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(gbm)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

######################

**RandomForestClassifier**
######################

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Random Forest 모델 정의
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 모델 학습
rf_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred_rf = rf_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, zero_division=1)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest - Accuracy: {accuracy_rf:.3f}")
print(f"Random Forest - Precision: {precision_rf:.3f}")
print(f"Random Forest - Recall: {recall_rf:.3f}")
print(f"Random Forest - F1 Score: {f1_rf:.3f}")

X_train shape: (607761, 60, 62)
X_test shape: (151941, 60, 62)
y_train shape: (607761,)
y_test shape: (151941,)
Random Forest - Accuracy: 0.947
Random Forest - Precision: 0.966
Random Forest - Recall: 0.927
Random Forest - F1 Score: 0.946


In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(rf_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

######################

**GradientBoostingClassifier**
######################

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# Gradient Boosting 모델 정의
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42)

# 모델 학습
gb_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred_gb = gb_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_gb = accuracy_score(y_test, y_pred_gb)
precision_gb = precision_score(y_test, y_pred_gb, zero_division=1)
recall_gb = recall_score(y_test, y_pred_gb)
f1_gb = f1_score(y_test, y_pred_gb)

print(f"Gradient Boosting - Accuracy: {accuracy_gb:.3f}")
print(f"Gradient Boosting - Precision: {precision_gb:.3f}")
print(f"Gradient Boosting - Recall: {recall_gb:.3f}")
print(f"Gradient Boosting - F1 Score: {f1_gb:.3f}")

In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(gb_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

######################

**AdaBoostClassifier**
######################

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# AdaBoost 모델 정의
ab_model = AdaBoostClassifier(n_estimators=100, learning_rate=0.05, random_state=42)

# 모델 학습
ab_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred_ab = ab_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_ab = accuracy_score(y_test, y_pred_ab)
precision_ab = precision_score(y_test, y_pred_ab, zero_division=1)
recall_ab = recall_score(y_test, y_pred_ab)
f1_ab = f1_score(y_test, y_pred_ab)

print(f"AdaBoost - Accuracy: {accuracy_ab:.3f}")
print(f"AdaBoost - Precision: {precision_ab:.3f}")
print(f"AdaBoost - Recall: {recall_ab:.3f}")
print(f"AdaBoost - F1 Score: {f1_ab:.3f}")

In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(ab_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

######################

**ExtraTreesClassifier**
######################

In [ ]:
#신규 ExtraTreesClassifier 모델 학습 및 예측 및 결과 검증

from sklearn.ensemble import ExtraTreesClassifier

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# Extra Trees 모델 정의
et_model = ExtraTreesClassifier(n_estimators=100, random_state=42)

# 모델 학습
et_model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

# 예측
y_pred_et = et_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_et = accuracy_score(y_test, y_pred_et)
precision_et = precision_score(y_test, y_pred_et, zero_division=1)
recall_et = recall_score(y_test, y_pred_et)
f1_et = f1_score(y_test, y_pred_et)

print(f"Extra Trees - Accuracy: {accuracy_et:.3f}")
print(f"Extra Trees - Precision: {precision_et:.3f}")
print(f"Extra Trees - Recall: {recall_et:.3f}")
print(f"Extra Trees - F1 Score: {f1_et:.3f}")

# 교차 검증 평가
X_reshaped = X_train.reshape(X_train.shape[0], -1)
accuracy_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='accuracy')
precision_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='precision')
recall_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='recall')
f1_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='f1')

print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

Extra Trees - Accuracy: 0.958
Extra Trees - Precision: 0.964
Extra Trees - Recall: 0.952
Extra Trees - F1 Score: 0.958


In [ ]:
# ExtraTreesClassifier 모델 저장
model_path = '/content/drive/MyDrive/Data/SO60_SMALL_ExtraTreesClassifier_model.joblib'
joblib.dump(et_model, model_path)

print(f"Model saved to {model_path}")

Model saved to /content/drive/MyDrive/Data/SO60_SMALL_ExtraTreesClassifier_model.joblib


In [ ]:
# ExtraTreesClassifier 모델 로딩
model_path_load = '/content/drive/MyDrive/Data/SO60_SMALL_ExtraTreesClassifier_model.joblib'

et_model = joblib.load(model_path_load)

print("Model loaded successfully")


In [ ]:
# 로드된 xtraTreesClassifier 모델 예측 및 결과 검증

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# 예측
y_pred_et = et_model.predict(X_test.reshape(X_test.shape[0], -1))

# 평가
accuracy_et = accuracy_score(y_test, y_pred_et)
precision_et = precision_score(y_test, y_pred_et, zero_division=1)
recall_et = recall_score(y_test, y_pred_et)
f1_et = f1_score(y_test, y_pred_et)

print(f"Extra Trees - Accuracy: {accuracy_et:.3f}")
print(f"Extra Trees - Precision: {precision_et:.3f}")
print(f"Extra Trees - Recall: {recall_et:.3f}")
print(f"Extra Trees - F1 Score: {f1_et:.3f}")

# 교차 검증 평가
X_reshaped = X_train.reshape(X_train.shape[0], -1)
accuracy_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='accuracy')
precision_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='precision')
recall_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='recall')
f1_cv = cross_val_score(et_model, X_reshaped, y_train, cv=5, scoring='f1')

print(f"Cross-Validated Accuracy: {np.mean(accuracy_cv):.3f}")
print(f"Cross-Validated Precision: {np.mean(precision_cv):.3f}")
print(f"Cross-Validated Recall: {np.mean(recall_cv):.3f}")
print(f"Cross-Validated F1 Score: {np.mean(f1_cv):.3f}")

In [ ]:
# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# SHAP 값 계산 및 시각화
explainer = shap.Explainer(et_model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)

In [ ]:
#신규 데이터 예측

# 예측 수행
predictions = et_model.predict(df_new_data)

# 예측 결과를 원래 데이터프레임에 추가
df_new_data['predictions'] = predictions

# 결과 출력
print(df_new_data)

######################

**tensorflow**
######################

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# 학습 데이터와 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# DNN 모델 정의
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

# 모델 저장
model.save('dnn_model.h5')

# 모델 로드
loaded_model = tf.keras.models.load_model('dnn_model.h5')

# 로드된 모델로 예측
y_pred_loaded = (loaded_model.predict(X_test) > 0.5).astype("int32")

# 로드된 모델 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
precision_loaded = precision_score(y_test, y_pred_loaded)
recall_loaded = recall_score(y_test, y_pred_loaded)
f1_loaded = f1_score(y_test, y_pred_loaded)

print(f"Loaded DNN - Accuracy: {accuracy_loaded:.3f}")
print(f"Loaded DNN - Precision: {precision_loaded:.3f}")
print(f"Loaded DNN - Recall: {recall_loaded:.3f}")
print(f"Loaded DNN - F1 Score: {f1_loaded:.3f}")

In [ ]:
# SHAP 값 계산 및 시각화
explainer = shap.Explainer(model)
shap_values = explainer(X_test.reshape(X_test.shape[0], -1))
shap.summary_plot(shap_values, X_test.reshape(X_test.shape[0], -1), feature_names=X.columns)